In [1]:
!git clone https://github.com/PonyGE/PonyGE2.git

Cloning into 'PonyGE2'...


In [2]:
!cd PonyGE2/ && git pull

Already up to date.


In [16]:
import shutil

experiment = "regression"

shutil.move(f"./fitness/{experiment}.py", "./PonyGE2/src/fitness/")


'./PonyGE2/src/fitness/regression.py'

In [17]:
!cd ./PonyGE2/src && python ponyge.py --parameters ../../parameters/{experiment}.txt
shutil.move(f"./PonyGE2/src/fitness/{experiment}.py", "./fitness/")


Start:	 2023-02-06 18:59:11.423582 

         Unit productions consume GE codons.
         Unit productions consume GE codons.
         Unit productions consume GE codons.
arima;srv
{'linear': {'arima': 1}, 'nlinear': {'srv': 1}}
arima;srv
{'linear': {'arima': 1}, 'nlinear': {'srv': 1}}
arima;mlp
{'linear': {'arima': 1}, 'nlinear': {'mlp': 1}}
arima;srv
{'linear': {'arima': 1}, 'nlinear': {'srv': 1}}
arima;srv
{'linear': {'arima': 1}, 'nlinear': {'srv': 1}}
arima;mlp
{'linear': {'arima': 1}, 'nlinear': {'mlp': 1}}
arima;mlp
{'linear': {'arima': 1}, 'nlinear': {'mlp': 1}}
arima;rbf
{'linear': {'arima': 1}, 'nlinear': {'rbf': 1}}
arima;srv
{'linear': {'arima': 1}, 'nlinear': {'srv': 1}}
arima;mlp
{'linear': {'arima': 1}, 'nlinear': {'mlp': 1}}
arima;rbf
{'linear': {'arima': 1}, 'nlinear': {'rbf': 1}}
arima;mlp
{'linear': {'arima': 1}, 'nlinear': {'mlp': 1}}
arima;rbf
{'linear': {'arima': 1}, 'nlinear': {'rbf': 1}}
arima;mlp
{'linear': {'arima': 1}, 'nlinear': {'mlp': 1}}
arima;mlp
{'lin

'./fitness/regression.py'